In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
%matplotlib inline

## 베이스라인 모델

In [2]:
is_mps = torch.backends.mps.is_available()
device = torch.device('mps' if is_mps else 'cpu')
# device = 'cpu'
print('current_device : ',device)

batch_size = 30
epochs = 30

current_device :  mps


In [3]:
import os
# os.mkdir('/Volumes/My Passport/dataset/data')
# !unzip -qq '/Volumes/My Passport/dataset/dataset.zip' -d '/Volumes/My Passport/dataset/data'

In [4]:
import os
#폴더 이미 생성되어 주석처리
original_dataset_dir = '/Volumes/My Passport/dataset/data'
classes_list = []

for i in os.listdir(original_dataset_dir):
    if os.path.isdir(os.path.join(original_dataset_dir,i)):
        classes_list.append(i)

base_dir = '/Volumes/My Passport/dataset/splitted'
# os.mkdir(base_dir)

In [5]:
#데이터 분기할 폴더 트리 생성
import shutil
train_dir = os.path.join(base_dir,'train')
# os.mkdir(train_dir)
validation_dir = os.path.join(base_dir,'validation')
# os.mkdir(validation_dir)
test_dir = os.path.join(base_dir,'test')
# os.mkdir(test_dir)

for cls in classes_list:
    print(cls)
    # os.mkdir(os.path.join(train_dir,cls))
    # os.mkdir(os.path.join(validation_dir,cls))
    # os.mkdir(os.path.join(test_dir,cls))

Pepper,_bell___healthy
Grape___Esca_(Black_Measles)
Pepper,_bell___Bacterial_spot
Strawberry___healthy
Grape___Black_rot
Corn___Common_rust
Apple___Apple_scab
Potato___healthy
Potato___Late_blight
Cherry___healthy
Tomato___Bacterial_spot
Apple___Black_rot
Cherry___Powdery_mildew
Corn___Cercospora_leaf_spot Gray_leaf_spot
Peach___healthy
Tomato___Early_blight
Tomato___Tomato_Yellow_Leaf_Curl_Virus
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
Potato___Early_blight
Grape___healthy
Apple___Cedar_apple_rust
Corn___Northern_Leaf_Blight
Tomato___Septoria_leaf_spot
Corn___healthy
Strawberry___Leaf_scorch
Tomato___Tomato_mosaic_virus
Tomato___Leaf_Mold
Tomato___Late_blight
Tomato___healthy
Tomato___Spider_mites Two-spotted_spider_mite
Apple___healthy
Tomato___Target_Spot
Peach___Bacterial_spot


In [6]:
# #데이터 현황 확인
# #신규 트리구조로 파일 복사
# import math

# for cls in classes_list:
#     if os.path.isdir(os.path.join(original_dataset_dir, cls)):
#         path = os.path.join(original_dataset_dir,cls)
#         #DS.store 파일 무시
#         fnames = sorted((f for f in os.listdir(path) if not f.startswith('.')),key=str.lower)
#         print(len(fnames))
#         train_size = math.floor(len(fnames) * 0.6)
#         validation_size = math.floor(len(fnames) * 0.2)
#         test_size = math.floor(len(fnames) * 0.2)
        
#         train_fnames = fnames[:train_size]
#         print("Train size(",cls,") ", len(train_fnames))
#         for fname in train_fnames:
#             source = os.path.join(path,fname)
#             destination = os.path.join(os.path.join(train_dir,cls),fname)
#             shutil.copyfile(source,destination)
        
#         validation_fnames = fnames[train_size:(train_size+validation_size)]
#         print("Validation size(",cls,") ", len(validation_fnames))
#         for fname in validation_fnames:
#             source = os.path.join(path,fname)
#             destination = os.path.join(os.path.join(validation_dir,cls),fname)
#             shutil.copyfile(source,destination)
        
#         test_fnames = fnames[(train_size+validation_size):(validation_size+train_size+test_size)]
#         print("Test size(",cls,") ", len(test_fnames))
#         for fname in test_fnames:
#             source = os.path.join(path,fname)
#             destination = os.path.join(os.path.join(test_dir,cls),fname)
#             shutil.copyfile(source,destination)
        
        

In [7]:
#dsfile 삭제
folders = [train_dir,validation_dir,test_dir]

for folder in folders:
    for cls in classes_list:
        target = folder+'/'+cls
        for root, dirs, files in os.walk(target):
            for file in files:
                if file.startswith('.'):
                    os.remove(os.path.join(root, file))


In [8]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

#이미지 사이즈 조정 및 텐서화
transform_base = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()])
#이미지폴더 : 폴더를 클래스화. 폴더 이름을 라벨로 지정
train_dataset = ImageFolder(root=train_dir,transform=transform_base)
validation_dataset = ImageFolder(root=validation_dir,transform=transform_base)

In [9]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset,
                          batch_size = batch_size,
                          shuffle = True,
                          num_workers = 4
                          )
val_loader = DataLoader(validation_dataset,
                          batch_size = batch_size,
                          shuffle = True,
                          num_workers = 4
                          )

In [10]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()

        self.conv1 = nn.Conv2d(3,32,3,padding=1)
        self.conv2 = nn.Conv2d(32,64,3,padding=1)
        self.conv3 = nn.Conv2d(64,64,3,padding=1)
        
        self.pool = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(4096,512)
        self.fc2 = nn.Linear(512,33)

    def forward(self,x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x,p=0.25,training=self.training)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x,p=0.25,training=self.training)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x,p=0.25,training=self.training)

        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x,p=0.5,training=self.training)
        x = self.fc2(x)

        return F.log_softmax(x,dim=1)


In [11]:
model_base = Net().to(device)
optimizer = optim.Adam(model_base.parameters(),lr=0.001)

In [12]:
def train(model, train_loader,optimizer):
    model.train()
    for batch_idx, (data,target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output,target)
        loss.backward()
        optimizer.step()

In [13]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data,target = data.to(device),target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output,target,reduction='sum').sum()
            pred = output.max(1,keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct/len(test_loader.dataset)
    return test_loss,test_accuracy

In [14]:
import time
import copy

def train_baseline(model, train_loader, val_loader, optimizer, num_epochs=30):
    best_acc = 0.0
    best_model_weight = copy.deepcopy(model.state_dict())

    for epoch in range(1,num_epochs+1):
        since = time.time()
        train(model, train_loader,optimizer)
        train_loss,train_acc = evaluate(model,train_loader)
        val_loss, val_acc = evaluate(model, val_loader)

        if val_acc > best_acc:
            best_acc = val_acc
            best_model_weight = copy.deepcopy(model.state_dict())
        
        time_elapsed = time.time() - since
        print('------------- epoch {} -------------'.format(epoch))
        print('train loss:{:.4f}, Accuracy:{:.2f}%'.format(train_loss,train_acc))
        print('val loss:{:.4f}, Accuracy:{:.2f}%'.format(val_loss,val_acc))
        print('completed {:.0f}m {:.0f}s'.format(time_elapsed//60,time_elapsed%60))
    
    model.load_state_dict(best_model_weight)
    return model


추론을 위해 모델을 저장할 때는 그 모델의 학습된 매개변수만 저장하면 됩니다. torch.save() 를 사용하여 모델의 state_dict 를 저장하는 것이 나중에 모델을 사용할 때 가장 유연하게 사용할 수 있는, 모델 저장 시 권장하는 방법입니다.

PyTorch에서는 모델을 저장할 때 .pt 또는 .pth 확장자를 사용하는 것이 일반적인 규칙입니다.

추론을 실행하기 전에 반드시 model.eval() 을 호출하여 드롭아웃 및 배치 정규화를 평가 모드로 설정하여야 합니다. 이 과정을 거치지 않으면 일관성 없는 추론 결과가 출력됩니다.

In [15]:

base = train_baseline(model_base,train_loader,val_loader,optimizer,epochs)
torch.save(base,'badseline.pt')

------------- epoch 1 -------------
train loss:0.8813, Accuracy:74.68%
val loss:0.9231, Accuracy:73.34%
completed 2m 20s
------------- epoch 2 -------------
train loss:0.6885, Accuracy:77.99%
val loss:0.7428, Accuracy:76.76%
completed 2m 18s
------------- epoch 3 -------------
train loss:0.4274, Accuracy:87.40%
val loss:0.4882, Accuracy:85.12%
completed 2m 19s
------------- epoch 4 -------------
train loss:0.3710, Accuracy:88.02%
val loss:0.4549, Accuracy:85.35%
completed 2m 18s
------------- epoch 5 -------------
train loss:0.3495, Accuracy:88.63%
val loss:0.4453, Accuracy:85.22%
completed 2m 17s
------------- epoch 6 -------------
train loss:0.2308, Accuracy:93.18%
val loss:0.3342, Accuracy:89.40%
completed 18m 53s
------------- epoch 7 -------------
train loss:0.2068, Accuracy:93.84%
val loss:0.3243, Accuracy:89.47%
completed 10m 4s
------------- epoch 8 -------------
train loss:0.1604, Accuracy:95.11%
val loss:0.2802, Accuracy:90.93%
completed 21m 9s
------------- epoch 9 ---------

## 전이학습 모델

In [ ]:
data_transforms = {
    'train' : transforms.Compose([transforms.Resize([64,64]),
        transforms.RandomHorizontalFlip(),transforms.RandomVerticalFlip(),
        
                                  ])

}